![](images/6.3.7.jpg)

In [1]:
import numpy as np
from sympy import *
from numpy.linalg import inv,det
def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(Number)})

Plain strain element coords of nodes $ {}^0x^k $:

In [2]:
x0 = np.array([[0.2,0.0,0.0,0.2],[0.2,0.2,0.0,0.0]])

Deformation gradient $ {}^t_0\textbf{X}_{ij}$ : $$ {}^t_0\textbf{X}_{ij} ={}^t_0x_{i,j} = \frac{\partial {}^tx_i}{\partial {}^0x_j} $$

In [3]:
X= np.array([[1.0,0.5],[0,1.5]])
xt = X.dot(x0)

$$  \textbf{C} =
\frac{E}{(1+\nu)(1-2\nu)} \left(\begin{array}{cc} 
1-\nu & \nu & 0\\
\nu & 1-\nu & 0 \\
0 & 0 & \frac{1-2\nu}{2}
\end{array}\right)
$$ 

In [4]:
E = symbols('E')
v=0.3
stiffness_m = E/((1+v)*(1-2*v))*np.array([[1-v,v,0],[v,1-v,0],[0,0,(1-2*v)/2]])
print(stiffness_m)

[[1.34615384615385*E 0.576923076923077*E 0]
 [0.576923076923077*E 1.34615384615385*E 0]
 [0 0 0.384615384615385*E]]


Interpolation functions of form: 
$$ h_1 = \frac{1}{4}(1+r)(1+s)$$
$h_{1,1}$ is the derivative w.r.t. $\frac{\partial}{\partial^0x_1}$ :
$$h_{1,1}=\frac{\partial h_1}{\partial r}\frac{\partial r}{\partial^0x_1} = \frac{5}{2}(1+s), \frac{\partial r}{\partial^0x_1} = 10$$
resp.:
$$h_{1,2}=\frac{\partial h_1}{\partial s}\frac{\partial s}{\partial^0x_2} = \frac{5}{2}(1+r), \frac{\partial s}{\partial^0x_2} = 10$$

In [5]:
x, y = symbols('x y')
scale=[1/10,1/10];
def h1(x,y): return 1/4*(1+x)*(1+y)
def h2(x,y): return 1/4*(1-x)*(1+y)
def h3(x,y): return 1/4*(1-x)*(1-y)
def h4(x,y): return 1/4*(1+x)*(1-y)
def hprime(h,x,y,j):
    dim =[x,y]
    return diff(h(x,y),dim[j]) / scale[j]
list_of_interp_fcts = np.array([h1,h2,h3,h4])

$$ {}^t_0\textbf{B}_{L0} = \begin{bmatrix}
h_{1,1} & 0 &|& ...\\
0 &h_{1,2} &|& ...\\
h_{1,2} & h_{1,1} &|& ...
\end{bmatrix} $$

In [6]:
BL = np.array([]).reshape(0,3)
for hi in list_of_interp_fcts:
    BL=np.concatenate((BL,np.array([
        [hprime(hi,x,y,0),0,hprime(hi,x,y,1)],
        [0,hprime(hi,x,y,1),hprime(hi,x,y,0)]
])))
BL =BL.T
print(BL[0])

[2.5*y + 2.5 0 -2.5*y - 2.5 0 2.5*y - 2.5 0 -2.5*y + 2.5 0]


$$ {}^tu^k = {}^tx^k - {}^0x^k $$

In [7]:
u = xt-x0
print(u)

[[0.1 0.1 0.  0. ]
 [0.1 0.1 0.  0. ]]


$$l_{ij} = \sum_{k=1}^{N} h_{k,j} {}^tu_i^k$$

In [8]:
def lij(u,n,i,j):
    h=[h1,h2,h3,h4]
    lsum=0
    for k in range(n):
        lsum = lsum + hprime(h[k],x,y,j)*u[i][k]
    return lsum

$$ {}^t_0\textbf{B}_{L1} = \begin{bmatrix}
l_{11}h_{1,1} & l_{21}h_{1,1} &|& ...\\
l_{12}h_{1,2} & l_{22}h_{1,2} &|& ...\\
l_{11}h_{1,2}+l_{12}h_{1,1} & l_{21}h_{1,2}+l_{22}h_{1,1} &|& ...
\end{bmatrix} $$

In [9]:
BL1 = np.array([]).reshape(0,3)
for hi in list_of_interp_fcts:
    BL1=np.concatenate((BL1,np.array([
        [hprime(hi,x,y,0)*lij(u,4,0,0),hprime(hi,x,y,1)*lij(u,4,0,1),hprime(hi,x,y,1)*lij(u,4,0,0)+hprime(hi,x,y,0)*lij(u,4,0,1)],
        [hprime(hi,x,y,0)*lij(u,4,1,0),hprime(hi,x,y,1)*lij(u,4,1,1),hprime(hi,x,y,1)*lij(u,4,1,0)+hprime(hi,x,y,0)*lij(u,4,1,1)]
])))
BL1 =BL1.T

$$ {}^t_0\textbf{B}_L = {}^t_0\textbf{B}_{L0} + {}^t_0\textbf{B}_{L1} $$

In [10]:
BL=BL+BL1

$$ J = \frac{\partial^0x_1}{\partial r}\frac{\partial^0x_2}{\partial s} =\frac{1}{10^2} $$

$$ {}^t_0\textbf{K}_L = \int_{-1}^{1} \int_{-1}^{1} {}^t_0\textbf{B}^T_{L}\textbf{C}{}_0^t\textbf{B}_{L}\frac{1}{10^2} dr ds $$

In [11]:
stiffness_linear = BL.T.dot(stiffness_m).dot(BL)*scale[0]*scale[1]


In [12]:
K = np.array([]).reshape(0,8);
for row in stiffness_linear:
    nodal_row = []
    for element in row:
        integration_over_volume= integrate(integrate(element,(x,-1,1)),(y,-1,1))
        nodal_row.append(integration_over_volume)
    K = np.concatenate((K,np.array([nodal_row])));
print(round_expr(K[0][0],4))
K =K.T

0.9615*E


$$ {}^t_0\textbf{B}_{NL} = \begin{bmatrix}
h_{1,1} & 0 &|& ...\\
h_{1,2} & 0 &|& ...\\
0 & h_{1,1} &|& ...\\
0 & h_{1,2} &|& ...
\end{bmatrix} $$

In [13]:
BNL = np.array([]).reshape(0,4)
for hi in list_of_interp_fcts:
    BNL=np.concatenate((BNL,np.array([
        [hprime(hi,x,y,0),hprime(hi,x,y,1),0,0],
        [0,0,hprime(hi,x,y,0),hprime(hi,x,y,1)]
    ])))
BNL =BNL.T

In [14]:
cauchy_stress = pow(10,7) * np.array([[5.849,6.971],[6.971,15.14]])
print(cauchy_stress)

[[5.849e+07 6.971e+07]
 [6.971e+07 1.514e+08]]


$$ {}^t_0\textbf{S} = det({}_0^t\textbf{X}){}^0_t\textbf{X} {}^t\mathbf{\sigma} {}^0_t\textbf{X}^T  $$

In [15]:
X0 = inv(X)
piola_stress = det(X)* (X0).dot(cauchy_stress).dot(X0.T)
print(piola_stress )

[[4.32583333e+07 1.92433333e+07]
 [1.92433333e+07 1.00933333e+08]]


In [16]:
exploded_piola = np.array([
    [piola_stress[0][0], piola_stress[0][1], 0,0],
    [piola_stress[0][1], piola_stress[1][1], 0,0],
    [0,0,piola_stress[0][0], piola_stress[0][1]],
    [0,0,piola_stress[0][0], piola_stress[0][1]]
    ])

$$ J = \frac{\partial^0x_1}{\partial r}\frac{\partial^0x_2}{\partial s} =\frac{1}{10^2} $$

$$ {}^t_0\textbf{K}_{NL} = \int_{-1}^{1} \int_{-1}^{1} {}^t_0\textbf{B}^T_{NL}\textbf{S}_{exp}{}_0^t\textbf{B}_{NL}\frac{1}{10^2} dr ds $$

In [17]:
stiffness_non_linear = BNL.T.dot(exploded_piola).dot(BNL) *scale[0]*scale[1]

In [18]:
K_NL = np.array([]).reshape(0,8);
for row in stiffness_non_linear:
    nodal_row = []
    for element in row:
        integration_over_volume= integrate(integrate(element,(x,-1,1)),(y,-1,1))
        nodal_row.append(integration_over_volume)
    K_NL = np.concatenate((K_NL,np.array([nodal_row])));
K_NL=K_NL.T

$$ {}^t_0\textbf{K} = {}^t_0\textbf{K}_{L} + {}^t_0\textbf{K}_{NL} $$

In [19]:
K=K + K_NL
print(round_expr(K[0][0],4))

0.9615*E + 57685555.5556
